In [4]:
import pandas as pd
from caveclient import CAVEclient
client = CAVEclient('minnie65_phase3_v1')
from datetime import datetime, timezone


In [5]:
###compare cg calls with Leila's classifier
settime = datetime(2022, 9, 20, 4, 0, 0, 0, tzinfo=timezone.utc)
md3 = client.materialize.query_table('allen_subclass_type_svm_v0', timestamp = settime)
df = client.materialize.query_table('cg_cell_type_calls', timestamp = settime)
df = df.drop_duplicates(subset ='post_pt_root_id')

df_comp = pd.merge(df, md3, left_on = 'post_pt_root_id', right_on = 'pt_root_id')

df_comp['cell_type_x'] = df_comp['cell_type_x'].str.replace('1P','23P')
df_comp['cell_type_x'] = df_comp['cell_type_x'].str.replace('5P_IT','5P-IT')
df_comp['cell_type_x'] = df_comp['cell_type_x'].str.replace('5P_NP','5P-NP')
df_comp['cell_type_x'] = df_comp['cell_type_x'].str.replace('5P_PT','5P-ET')                   
                   
df_comp['cell_type_y'] = df_comp['cell_type_y'].str.replace('BC','INH')
df_comp['cell_type_y'] = df_comp['cell_type_y'].str.replace('MC','INH')
df_comp['cell_type_y'] = df_comp['cell_type_y'].str.replace('NGC','INH')
df_comp['cell_type_y'] = df_comp['cell_type_y'].str.replace('BPC','INH')
df_comp['cell_type_y'] = df_comp['cell_type_y'].str.replace('6P-CT','6P')
df_comp['cell_type_y'] = df_comp['cell_type_y'].str.replace('6P-IT','6P')
df_comp['cell_type_y'] = df_comp['cell_type_y'].str.replace('5P-PT','5P-ET')
                   
df_comp_sub = df_comp[['cell_type_x','pt_root_id',  'cell_type_y' ]]
df_comp_sub = df_comp_sub.rename(columns={"cell_type_x": "Manual", "cell_type_y": "Predicted"})
table = pd.pivot_table(df_comp_sub, index=['Manual'],
                    columns=['Predicted'], aggfunc="count", fill_value=0)
print (table)

          pt_root_id                                
Predicted        23P   4P 5P-ET 5P-IT 5P-NP  6P  INH
Manual                                              
23P              545  419     1     1     0   0    1
4P                 5  492     1   133     0   0    5
5P-ET              0    0   112    20     0   3   19
5P-IT              1    8     5   234     2  41   15
5P-NP              0    0     0     1     0   0    0
6P                 0    0     0     0     0  50    0
INH                2    4     3     4     0   1  195
